# Data Exploration



## Import dependencies

In [30]:
import numpy as np
import pandas as pd
import sqlite3
import plotly.express as px
import matplotlib.pyplot as plt
import folium

## Load data

In [31]:
# Step 2: Load the datasets
conn = sqlite3.connect('traffic_violation.sqlite')
parking_df = pd.read_sql_query("SELECT * FROM parking_violations", conn)
street_df = pd.read_sql_query("SELECT * FROM street_directory", conn)

## Show the Unique values

In [32]:
# Assuming you have the data frames parking_df and street_df

# Extract unique values from parking_df['Crime_location']
parking_unique_values = set(parking_df['Crime_Str_No'].unique())

# Extract unique values from street_df['Street_Name']
street_unique_values = set(street_df['Street_Number'].unique())

# Find the unique values that are in both sets
common_values = parking_unique_values & street_unique_values

# Print the common values
print(len(common_values))
print(common_values)

704
{2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2058, 2059, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 2122, 2123, 2124, 2125, 2126, 2127, 2128, 2129, 2130, 2131, 2132, 2133, 2134, 2135, 2136, 2137, 2138, 2139, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2150, 2151, 2152, 2153, 2154, 2155, 2156, 2157, 2159, 2200, 2201, 2202, 2203, 2204, 2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223, 2224, 2225, 2226, 2227, 2228, 2229, 2230, 2231, 2232, 2233, 2234, 2235, 2236, 2237, 2238, 2239, 2240, 2242, 2243, 2244, 2245, 2246, 2247, 2248, 2249, 2250, 2251, 2253, 2254, 2255, 2256, 2257, 2258, 2259, 2300, 2301, 2302, 2303, 2304, 2305, 2306, 2307, 2310, 2311, 2312, 2314, 2315, 2316, 2317, 2318, 2319, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2328, 2329, 2330, 2331, 2332, 2333, 2334, 2335, 2336, 2337, 2338, 2339, 2340, 2341,

## Merge Datasets

In [33]:
# Step 3: Merge the datasets
merged_df = pd.merge(parking_df, street_df, left_on='Crime_Str_No', right_on='Street_Number', how='inner')
grouped_df = merged_df.groupby(['Crime_Str_No', 'Street_Number', 'Street_Name', 'latitude', 'longitude']).size().reset_index(name='Total Violations')
print(len(grouped_df))

704


In [34]:
print(merged_df)

       Crime_Str_No                                     Crime_location  \
0              1606                         Bonn, Lyngsbergstra√üe 108   
1              1606              Bonn, Siemensstra√üe Parkplatz Sirius   
2              1606         Bonn, Wilhelmstra√üe gegen√ºber Hnr. 40/42   
3              1606                  Bonn, F√ºrstenstra√üe Ecke Am Hof   
4              1606  Bonn, Obere Wilhelmstra√üe Ecke Beueler Bahnho...   
...             ...                                                ...   
47618          2340                           Friesdorfer Str. 248-250   
47619          2340                               Friesdorfer Str. 250   
47620          2340                               Friesdorfer Str. 252   
47621          2340                      Bonn, Villichgasse vor Hnr. 2   
47622          2346                        Bonn, Villichgasse ohne Nr.   

       Crime_Violation_No  Total_Fine Vehicle_Description  Street_Number  \
0                  112293          

### Data exploration
Print some basic information about the data. Your data exploration would continue here.

In [35]:
# Step 4: Create a scatter plot
fig = px.scatter(grouped_df, x='Street_Number', y='Street_Name', size='Total Violations', color='Total Violations')


In [36]:
# Step 5: Display the plot
fig.show()  # Use this for plotly


In [37]:
print(grouped_df.columns)

Index(['Crime_Str_No', 'Street_Number', 'Street_Name', 'latitude', 'longitude',
       'Total Violations'],
      dtype='object')


In [38]:
print(grouped_df)

     Crime_Str_No  Street_Number             Street_Name  latitude  longitude  \
0            1001           1001       Ernst-Abbe-Straﬂe  50.71579   7.115673   
1            1002           1002             Acherstraﬂe  50.71579   7.115673   
2            1003           1003  Achim-von-Arnim-Straﬂe  50.71579   7.115673   
3            1004           1004          Bergfeldstraﬂe  50.71579   7.115673   
4            1005           1005             Adolfstraﬂe  50.71579   7.115673   
..            ...            ...                     ...       ...        ...   
699          2354           2354      R¸desheimer Straﬂe  50.71579   7.115673   
700          2355           2355           R¸digerstraﬂe  50.71579   7.115673   
701          2356           2356      R¸ngsdorfer Straﬂe  50.71579   7.115673   
702          2358           2358           Sachsenstraﬂe  50.71579   7.115673   
703          2359           2359      Saint-Cloud-Straﬂe  50.71579   7.115673   

     Total Violations  
0  

In [39]:
# Sample data
data = grouped_df

# Convert data to DataFrame
df = pd.DataFrame(data)

# Specify the latitude and longitude to center the map
center_latitude = df['latitude'][0]
center_longitude = df['longitude'][0]

# Create a folium map and set its center location
m = folium.Map(location=[center_latitude, center_longitude], zoom_start=13)

# Add circles to the map
for index, row in df.iterrows():
    street_name = row['Street_Name']
    total_violations = row['Total Violations']
    
    # Create a pop-up message with street_name and Total Violations
    popup_message = f"Street Name: {street_name}<br>Total Violations: {total_violations}"
    
    # Add circle marker with pop-up to the map
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['Total Violations'] * 0.25,  # Adjust the size scaling factor as needed
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5,
        popup=popup_message
    ).add_to(m)

# Save the map to an HTML file
m.save('map.html')